哈摟，各位資料科學家大家好，我是來自於台北市"頭期款不足"的阿宏，我想買房子很久了，但是不知道要買哪個地方，想先從內政部不動產交易時價登入資料中先了解行情，我現在手邊蒐集了台北市Taipei(A_lvr_land_A.csv)、新北市New_Taipei(F_lvr_land_A.csv)、台中市Taichung(B_lvr_land_A.csv)、高雄市Kaohsiung(E_lvr_land_A.csv)共四份資料，請各位資料科學家幫幫忙分析了。

# 1.不動產資料檔案讀取並串接

資料分散在4個資料，先合併成一個資料方便後續分析，但是我發現有英文欄位跟中文欄位，留下中文刪除英文，並加入新的欄位用以分辨台北市Taipei、新北市New_Taipei、台中市Taichung、高雄市Kaohsiung

1. 利用Pandas中的pd.read_csv()分別讀取A_lvr_land_A.csv、B_lvr_land_A.csv、E_lvr_land_A.csv、F_lvr_land_A.csv
2. 刪除csv中第二列英文翻譯，The villages and towns urban district,transaction sign,land sector position building sector house number plate,land shifting total area square meter,....
3. 生成city欄位其中以地區分類台北市Taipei(A_lvr_land_A.csv)、新北市New_Taipei(F_lvr_land_A.csv)、台中市Taichung(B_lvr_land_A.csv)、高雄市Kaohsiung(E_lvr_land_A.csv)
4. 將以上四份資料運用pd.concat()串接

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#1.利用Pandas中的pd.read_csv()分別讀取A_lvr_land_A.csv、B_lvr_land_A.csv、E_lvr_land_A.csv、F_lvr_land_A.csv
Taipei = pd.read_csv('A_lvr_land_A.csv', low_memory=False)
New_Taipei = pd.read_csv('F_lvr_land_A.csv', low_memory=False)
Taichung = pd.read_csv('B_lvr_land_A.csv', low_memory=False)
Kaohsiung = pd.read_csv('E_lvr_land_A.csv', low_memory=False)
Taipei.head(3)

,鄉鎮市區,交易標的,土地區段位置建物區段門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,單價元平方公尺,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯
0,The villages and towns urban district,transaction sign,land sector position building sector house num...,land shifting total area square meter,the use zoning or compiles and checks,the non-metropolis land use district,non-metropolis land use,transaction year month and day,transaction pen number,shifting level,...,the unit price (NTD / square meter),the berth category,berth shifting total area square meter,the berth total price NTD,the note,serial number,NaN,NaN,NaN,NaN
1,文山區,房地(土地+建物)+車位,臺北市文山區忠順街二段85巷1~30號,26.97,住,NaN,NaN,1090406,土地1建物1車位2,四層,...,156583,坡道平面,40.7,0,含增建或未登記建物。;內含獨立產權車位，未包含在棟數內。,RPVOMLMKNHPFFAA18CA,110.91,9.63,11.09,NaN
2,文山區,房地(土地+建物),臺北市文山區福興路91~120號,10.32,住,NaN,NaN,1090317,土地1建物1車位0,三層,...,149558,NaN,0.0,0,含增建或未登記建物。;,RPPPMLMKNHPFFAA67CA,24.58,1.28,2.30,NaN


In [3]:
#2.刪除csv中第二列英文翻譯，The villages and towns urban district,transaction sign,land sector position building sector house number plate,land shifting total area square meter,....
Taipei = Taipei.drop(index=0,inplace=False)
New_Taipei = New_Taipei.drop(index=0,inplace=False)
Taichung = Taichung.drop(index=0,inplace=False)
Kaohsiung = Kaohsiung.drop(index=0,inplace=False)
Taipei.head(3)

,鄉鎮市區,交易標的,土地區段位置建物區段門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,單價元平方公尺,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯
1,文山區,房地(土地+建物)+車位,臺北市文山區忠順街二段85巷1~30號,26.97,住,NaN,NaN,1090406,土地1建物1車位2,四層,...,156583,坡道平面,40.7,0,含增建或未登記建物。;內含獨立產權車位，未包含在棟數內。,RPVOMLMKNHPFFAA18CA,110.91,9.63,11.09,NaN
2,文山區,房地(土地+建物),臺北市文山區福興路91~120號,10.32,住,NaN,NaN,1090317,土地1建物1車位0,三層,...,149558,NaN,0.0,0,含增建或未登記建物。;,RPPPMLMKNHPFFAA67CA,24.58,1.28,2.30,NaN
3,文山區,房地(土地+建物),臺北市文山區木柵路四段121~150號,39.64,其他,NaN,NaN,1090418,土地2建物1車位0,六層,...,181472,NaN,0.0,0,NaN,RPSPMLMKNHPFFAA97CA,78.80,4.15,8.76,NaN


In [4]:
#3. 生成city欄位其中以地區分類台北市Taipei(A_lvr_land_A.csv)、新北市New_Taipei(F_lvr_land_A.csv)、台中市Taichung(B_lvr_land_A.csv)、高雄市Kaohsiung(E_lvr_land_A.csv)
Taipei['city'] = '台北市'
New_Taipei['city'] = '新北市'
Taichung['city'] = '台中市'
Kaohsiung['city'] = '高雄市'
Taipei.head(3)

,鄉鎮市區,交易標的,土地區段位置建物區段門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,city
1,文山區,房地(土地+建物)+車位,臺北市文山區忠順街二段85巷1~30號,26.97,住,NaN,NaN,1090406,土地1建物1車位2,四層,...,坡道平面,40.7,0,含增建或未登記建物。;內含獨立產權車位，未包含在棟數內。,RPVOMLMKNHPFFAA18CA,110.91,9.63,11.09,NaN,台北市
2,文山區,房地(土地+建物),臺北市文山區福興路91~120號,10.32,住,NaN,NaN,1090317,土地1建物1車位0,三層,...,NaN,0.0,0,含增建或未登記建物。;,RPPPMLMKNHPFFAA67CA,24.58,1.28,2.30,NaN,台北市
3,文山區,房地(土地+建物),臺北市文山區木柵路四段121~150號,39.64,其他,NaN,NaN,1090418,土地2建物1車位0,六層,...,NaN,0.0,0,NaN,RPSPMLMKNHPFFAA97CA,78.80,4.15,8.76,NaN,台北市


In [5]:
#4. 將以上四份資料運用pd.concat()串接
df = pd.concat([Taipei, New_Taipei, Taichung, Kaohsiung])

In [6]:
df.dtypes 

鄉鎮市區              object
交易標的              object
土地區段位置建物區段門牌      object
土地移轉總面積平方公尺       object
都市土地使用分區          object
非都市土地使用分區         object
非都市土地使用編定         object
交易年月日             object
交易筆棟數             object
移轉層次              object
總樓層數              object
建物型態              object
主要用途              object
主要建材              object
建築完成年月            object
建物移轉總面積平方公尺       object
建物現況格局-房          object
建物現況格局-廳          object
建物現況格局-衛          object
建物現況格局-隔間         object
有無管理組織            object
總價元               object
單價元平方公尺           object
車位類別              object
車位移轉總面積(平方公尺)     object
車位總價元             object
備註                object
編號                object
主建物面積            float64
附屬建物面積           float64
陽台面積             float64
電梯                object
city              object
dtype: object

# 2. 資料清理與切片

因為我是想買來住的房子，所以幫忙刪除住宿用途以外的資料，並且限制
1. 交易年月日，限制在109年
2. 建物現況格局-房，1~5間 
3. 建物現況格局-廳，1~2廳

以下columns_mapping、analysis_columns、columns_type提供給資料科學家配合使用
1. 利用.rename()並配合以下columns_mapping，將原中文欄位名稱改為英文方便之後分析
2. 取出主要用途(main_use)為'住家用'以及都市土地使用分區(use_zoning)為'住'的資料並針對以下欄位analysis_columns做分析並去除na值
   (提示:先取完之後再.dropna())
3. 觀察欄位資料型態，並利用.astype()搭配以下提供的columns_type做欄位型態轉換
4. 做資料切片將<br>
    新增欄位交易年月日(tx_dt_year)，從交易年月日(tx_dt)萃取出年份<br>
    1.交易年月日(tx_dt_year)，限制在109年<br>
    2.建物現況格局-房(room_number)，限制在1到5間<br>
    3.建物現況格局-廳(hall_number)，限制在1到2廳<br>
    4.最後運用.reset_index()重新定義索引

In [7]:
columns_mapping = {'鄉鎮市區':'towns',
'交易標的':'transaction_sign',
'土地區段位置建物區段門牌':'house_number',
'土地移轉總面積平方公尺':'land_area_square_meter', 
'都市土地使用分區':'use_zoning', 
'非都市土地使用分區':'land_use_district',
'非都市土地使用編定':'land_use',
'交易年月日':'tx_dt', 
 '交易筆棟數':'transaction_pen_number', 
 '移轉層次':'shifting_level', 
 '總樓層數':'total_floor_number', 
 '建物型態':'building_state', 
 '主要用途':'main_use', 
 '主要建材':'main_materials',
 '建築完成年月':'complete_date', 
 '建物移轉總面積平方公尺':'building_area_square_meter', 
 '建物現況格局-房':'room_number', 
 '建物現況格局-廳':'hall_number', 
 '建物現況格局-衛':'health_number', 
'建物現況格局-隔間':'compartmented_number', 
 '有無管理組織':'manages', 
 '總價元':'total_price', 
 '單價元平方公尺':'unit_price', 
 '車位類別':'berth_category', 
 '車位移轉總面積(平方公尺)':'berth_area_square_meter',
'車位總價元':'berth_price', 
 '備註':'note', 
 '編號':'serial_number', 
 '主建物面積':'main_building_area', 
 '附屬建物面積':'auxiliary_building_area', 
 '陽台面積':'balcony_area', 
 '電梯':'elevator'
                  }
analysis_columns = ['city','towns','main_use','use_zoning','total_price','building_area_square_meter',
                                     'main_building_area',
                                     'tx_dt','unit_price','room_number','hall_number','health_number']
columns_type = {'total_price': 'int','unit_price':'float','building_area_square_meter':'float',
                                      'main_building_area': 'float',
                                      'room_number': 'int','hall_number': 'int','health_number': 'int'}

In [8]:
#1. 利用.rename()並配合以下columns_mapping，將原中文欄位名稱改為英文方便之後分析
columns_mapping_df = df.rename(mapper=columns_mapping, axis=1)
columns_mapping_df.info

<bound method DataFrame.info of       towns transaction_sign         house_number land_area_square_meter  \
1       文山區     房地(土地+建物)+車位  臺北市文山區忠順街二段85巷1~30號                  26.97   
2       文山區        房地(土地+建物)     臺北市文山區福興路91~120號                  10.32   
3       文山區        房地(土地+建物)  臺北市文山區木柵路四段121~150號                  39.64   
4       中正區        房地(土地+建物)   臺北市中正區和平西路一段31~60號                   4.26   
5       文山區        房地(土地+建物)    臺北市文山區樟新街56巷1~30號                  44.57   
...     ...              ...                  ...                    ...   
14129   三民區        房地(土地+建物)       建興路380巷1弄1~30號                   32.0   
14130   三民區     房地(土地+建物)+車位             民族巷1~30號                   7.01   
14131   三民區     房地(土地+建物)+車位         大豐一路331~360號                  27.23   
14132   三民區     房地(土地+建物)+車位            民族巷31~60號                  10.32   
14133   楠梓區        房地(土地+建物)            青埔街61~90號                  12.61   

      use_zoning land_use_district land_use    tx_dt tr

In [14]:
#2. 取出主要用途(main_use)為'住家用'以及都市土地使用分區(use_zoning)為'住'的資料並針對以下欄位analysis_columns做分析並去除na值 (提示:先取完之後再.dropna())
#   (提示:先取完之後再.dropna())
columns_mapping_df = pd.DataFrame(columns_mapping_df.loc[columns_mapping_df['main_use'] == '住家用'])
columns_mapping_df = pd.DataFrame(columns_mapping_df.loc[columns_mapping_df['use_zoning'] == '住'])
columns_mapping_df_1 = columns_mapping_df
analysis_columns_df = columns_mapping_df[analysis_columns]
analysis_columns_df['tx_dt_year'] = analysis_columns_df['tx_dt'].str[:-4].astype(int)
analysis_columns_df_1 = analysis_columns_df
analysis_columns_df.isna().sum()
analysis_columns_df = pd.DataFrame(analysis_columns_df.dropna())
analysis_columns_df.isna().sum()
print(analysis_columns_df.shape, analysis_columns_df_1.shape, columns_mapping_df_1.shape)

(7802, 13) (7818, 13) (7818, 33)


C:\Users\user\anaconda3\envs\37env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,city,towns,main_use,use_zoning,total_price,building_area_square_meter,main_building_area,tx_dt,unit_price,room_number,hall_number,health_number,tx_dt_year
1,台北市,文山區,住家用,住,36000000,229.91,110.91,1090406,156583,3,2,2,109
2,台北市,文山區,住家用,住,5580000,37.31,24.58,1090317,149558,2,1,1,109
5,台北市,文山區,住家用,住,18280000,123.66,98.74,1090328,147825,3,2,2,109


In [10]:
#3. 觀察欄位資料型態，並利用.astype()搭配以下提供的columns_type做欄位型態轉換
analysis_columns_df = analysis_columns_df.astype(columns_type)
#analysis_columns_df.info
#(analysis_columns_df.loc[(analysis_columns_df['hall_number'] < 6)]).shape

In [11]:
#4. 做資料切片將
#     新增欄位交易年月日(tx_dt_year)，從交易年月日(tx_dt)萃取出年份
#     1.交易年月日(tx_dt_year)，限制在109年
#     2.建物現況格局-房(room_number)，限制在1到5間
#     3.建物現況格局-廳(hall_number)，限制在1到2廳
#     4.最後運用.reset_index()重新定義索引



analysis_columns_df_2 = analysis_columns_df.loc[(analysis_columns_df['room_number']>0)&(analysis_columns_df['room_number']<6) ]
analysis_columns_df_2 = analysis_columns_df_2.loc[(analysis_columns_df_2['hall_number']>0)&(analysis_columns_df_2['hall_number']<3) ]
analysis_columns_df_2 = analysis_columns_df_2.loc[(analysis_columns_df_2['tx_dt_year'] == 109 ) ]
analysis_columns_df_2.reset_index(inplace=False)
analysis_columns_df_2.head(3)

KeyError: 'tx_dt_year'

# 3. 建立自定義特徵加入分析


以台灣來說大家都是以坪為單位計算使用面積，應該是不會問說你家有幾平方公尺吧?，但是偏偏資料中沒有以坪為單位計算，所以接下來請各位幫忙產生新的欄位以坪為單位計算面積，轉換公式我也幫你找好了，在下面的定義。<br>
定義 : 1平方公尺相當於0.3025坪 <br>
建立新特徵
1. 建物移轉總面積坪(building_area_square_feet) : 建物移轉總面積平方公尺\*0.3025
2. 主建物面積坪(main_building_area_square_feet) : 主建物面積\*0.3025
3. 單價元坪(unit_price_square_feet) : 單價元平方公尺/0.3025

可以利用.describe()做一下資料觀察，是否有奇怪的資料?<br>
如果有請將資料移除，並說明為什麼移除此資料?<br>
移除後再用.describe()觀察是否還有奇怪的資料?

In [ ]:
# 建立新特徵
# 1. 建物移轉總面積坪(building_area_square_feet) : 建物移轉總面積平方公尺*0.3025
# 2. 主建物面積坪(main_building_area_square_feet) : 主建物面積*0.3025
# 3. 單價元坪(unit_price_square_feet) : 單價元平方公尺/0.3025
analysis_columns_df_2['building_area_square_feet'] = analysis_columns_df_2['building_area_square_meter']*0.3025
analysis_columns_df_2['main_building_area_square_feet'] = analysis_columns_df_2['main_building_area']*0.3025
analysis_columns_df_2['unit_price_square_feet'] = analysis_columns_df_2['unit_price']/0.3025
analysis_columns_df_2.head(3)

In [ ]:
# 可以利用.describe()做一下資料觀察，是否有奇怪的資料?
# 如果有請將資料移除，並說明為什麼移除此資料?

analysis_columns_df_2.describe()
analysis_columns_df_2 = analysis_columns_df_2.loc[(analysis_columns_df_2['main_building_area']>0)]
analysis_columns_df_2 = analysis_columns_df_2.loc[(analysis_columns_df_2['unit_price']>0)]
analysis_columns_df_2 = analysis_columns_df_2.loc[(analysis_columns_df_2['main_building_area_square_feet']>0)]
analysis_columns_df_2 = analysis_columns_df_2.loc[(analysis_columns_df_2['unit_price_square_feet']>0)]
print('main_building_area及unit_price為0的資料不合理')

In [ ]:
# 移除後再用.describe()觀察是否還有奇怪的資料?

analysis_columns_df_2.describe()

# 4. 找出台北市時價登入總價高度相關的變數

阿宏我是台北人他想找出影響台北市總價、單價元坪的因子<br>
相關係數0.3以下為低相關，0.3\~0.7為中等相關，0.7以上為高度相關<br>
1. 資料切片切出city欄位為台北市的資料，並找出時價登入總價(total_price)高度相關的變數
2. 資料切片切出city欄位為台北市的資料，找出單價元坪(unit_price_square_feet)高度相關的變數

In [ ]:
# 阿宏我是台北人他想找出影響台北市總價、單價元坪的因子
# 1. 資料切片切出city欄位為台北市的資料，並找出時價登入總價(total_price)高度相關的變數
str_choice = "台北市"
analysis_columns_df_Taipei = analysis_columns_df_2[analysis_columns_df_2['city'].str.contains(str_choice)]
total_price = analysis_columns_df_Taipei.corr() 
total_price_1 = total_price.loc[(total_price['total_price']>0.7)&(total_price['total_price']<1)]
# 2. 資料切片切出city欄位為台北市的資料，找出單價元坪(unit_price_square_feet)高度相關的變數
unit_price_square_feet = analysis_columns_df_Taipei.corr() 
unit_price_square_feet_1 = unit_price_square_feet.loc[(unit_price_square_feet['unit_price_square_feet']>0.7)&(unit_price_square_feet['unit_price_square_feet']<1)]
print(total_price_1['total_price'], unit_price_square_feet_1['unit_price_square_feet'])
unit_price_square_feet


f, ax = plt.subplots(figsize=(10, 7.5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
sns.heatmap(total_price, vmax=1, square=True, annot=True, annot_kws={'size': 10})

# 5. 資料視覺化並解釋


我想要以視覺化的方式來看房價資料，並且請各位資料科學家解釋圖表給阿宏我知道
1. 以城市(city)為x軸，以單價元坪(unit_price_square_feet)為y軸畫出boxplot，並找出單價元坪(unit_price_square_feet)中位數最高的地區
2. 進一步對台北市的資料做圖，以建物現況格局-房(room_number)為x軸，以總價元(total_price)為y軸畫出boxplot，並找出總價元(total_price)中位數最高的房間數。hint:資料切片找出city欄位為台北市的資料，再進一步畫圖
3. 對台北市的資料做圖，先將地區(twons)做編碼在進行，再以地區(twon)為x軸，以單價元坪(unit_price_square_feet)為y軸畫出boxplot，並找出單價元坪(unit_price_square_feet)中位數最高的地區。hint:運用LabelEncoder()對地區(twons)做編碼，運用.inverse_transform()反查編碼的地區

In [ ]:
#1. 以城市(city)為x軸，以單價元坪(unit_price_square_feet)為y軸畫出boxplot，並找出單價元坪(unit_price_square_feet)中位數最高的地區
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False

f, ax = plt.subplots(figsize=(10, 7.5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
sns.boxplot(x = analysis_columns_df_Taipei['city'],
            y = analysis_columns_df_Taipei['unit_price_square_feet'])
plt.ylabel('city', fontsize = 13)
plt.xlabel('unit_price_square_feet', fontsize = 13)
plt.show()

analysis_columns_df_Taipei['unit_price_square_feet'].median()

In [ ]:
f, ax = plt.subplots(figsize=(10, 7.5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
sns.boxplot(x = analysis_columns_df_Taipei['towns'],
            y = analysis_columns_df_Taipei['unit_price_square_feet'])
plt.ylabel('單價元坪', fontsize = 13)
plt.xlabel('鄉鎮區', fontsize = 13)
plt.show()

In [ ]:
#2. 進一步對台北市的資料做圖，以建物現況格局-房(room_number)為x軸，以總價元(total_price)為y軸畫出boxplot，並找出總價元(total_price)中位數最高的房間數。hint:資料切片找出city欄位為台北市的資料，再進一步畫圖
f, ax = plt.subplots(figsize=(10, 7.5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
sns.boxplot(x = analysis_columns_df_Taipei['room_number'],
            y = analysis_columns_df_Taipei['total_price'])
plt.ylabel('總價', fontsize = 13)
plt.xlabel('建物現況格局-房', fontsize = 13)
plt.show()

In [ ]:
#3. 對台北市的資料做圖，先將地區(twons)做編碼在進行，再以地區(twon)為x軸，以單價元坪(unit_price_square_feet)為y軸畫出boxplot，並找出單價元坪(unit_price_square_feet)中位數最高的地區。
# hint:運用LabelEncoder()對地區(twons)做編碼，運用.inverse_transform()反查編碼的地區
from sklearn.preprocessing import LabelEncoder




uniques =analysis_columns_df_Taipei['towns'].value_counts().index
counts = analysis_columns_df_Taipei['towns'].value_counts().values
plt.figure(figsize=(10, 10))
plt.bar(uniques, counts)
plt.xticks(fontsize=12)
plt.show()


In [ ]:

labelencoder = LabelEncoder()
labelencoder.fit_transform(analysis_columns_df_Taipei['towns'])

f, ax = plt.subplots(figsize=(10, 7.5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
sns.boxplot(x = analysis_columns_df_Taipei['towns'],
            y = analysis_columns_df_Taipei['unit_price_square_feet'])
plt.ylabel('單價元坪', fontsize = 13)
plt.xlabel('鄉鎮區', fontsize = 13)
plt.show()